In [105]:
from urllib.request import urlopen      ## WEB에서 URL기반으로 다운로드
from bs4 import BeautifulSoup           ## WEB데이터 객체화 및 정보 추출용 모듈
import pandas as pd

In [106]:
## 다운로드 받을 웹 사이트 URL 
W_URL = 'https://search.naver.com/search.naver?where=nexearch&sm=tab_etc&qvt=0&query=%ED%98%84%EC%9E%AC%EC%83%81%EC%98%81%EC%98%81%ED%99%94'

## 다운로딩
retObj = urlopen(W_URL)

## markup str ==> 객체화
bs = BeautifulSoup(retObj.read(), 'html.parser')


In [107]:
bs

<!DOCTYPE html>
 <html lang="ko"><head> <meta charset="utf-8"/> <meta content="strict-origin-when-cross-origin" name="referrer"/> <meta content="telephone=no,address=no,email=no" name="format-detection"/> <meta content="현재상영영화 : 네이버 검색" property="og:title"> <meta content="https://ssl.pstatic.net/sstatic/search/common/og_v3.png" property="og:image"/> <meta content="'현재상영영화'의 네이버 검색 결과입니다." property="og:description"/> <meta content="'현재상영영화'의 네이버 검색 결과입니다." lang="ko" name="description"/> <title>현재상영영화 : 네이버 검색</title> <link href="https://ssl.pstatic.net/sstatic/search/favicon/favicon_32x32_240820.ico" rel="shortcut icon"/> <link href="https://ssl.pstatic.net/sstatic/search/opensearch-description.https.xml" rel="search" title="Naver" type="application/opensearchdescription+xml"><script> if (top.frames.length!=0 || window!=top) window.open(location, "_top"); </script><link href="https://ssl.pstatic.net/sstatic/search/pc/css/search1_251023.css" rel="stylesheet" type="text/css"/> <link href=

In [110]:
movies_data = []

movies = bs.find_all('div', class_='data_area')

for movie in movies:
    # 제목
    title_tag = movie.find('a', class_='this_text _text')
    title = title_tag.get_text(strip=True) if title_tag else None

    # 개요, 상영시간, 개봉일, 평점, 출연
    info_groups = movie.find_all('dl', class_='info_group')
    genre, runtime, release, rating, actors = None, None, None, None, None

    for group in info_groups:
        dt_tag = group.find('dt')
        if not dt_tag:
            continue
        dt_text = dt_tag.get_text(strip=True)

        if dt_text == '개요':
            dds = group.find_all('dd')
            genre = dds[0].get_text(strip=True) if len(dds) > 0 else None
            runtime = dds[1].get_text(strip=True) if len(dds) > 1 else None
        elif dt_text == '개봉':
            dds = group.find_all('dd')
            release = dds[0].get_text(strip=True) if len(dds) > 0 else None
            rating_tag = group.find('span', class_='num')
            rating = rating_tag.get_text(strip=True) if rating_tag else None
        elif dt_text == '출연':
            actors_tag = group.find('span', class_='_text')
            actors = actors_tag.get_text(strip=True) if actors_tag else None

    movies_data.append({
        '제목': title,
        '장르': genre,
        '상영시간': runtime,
        '개봉일': release,
        '평점': rating,
        '출연': actors
    })

# DataFrame 변환 및 확인
df = pd.DataFrame(movies_data)
display(df)








,제목,장르,상영시간,개봉일,평점,출연
0,위키드: 포 굿,판타지,137분,2025.11.19.,7.54,"신시아 에리보, 아리아나 그란데, 조나단 베일리, 에단 슬레이터, 보웬 양, 마리사 보데"
1,나우 유 씨 미 3,범죄,112분,2025.11.12.,7.83,"제시 아이젠버그, 우디 해럴슨, 데이브 프랭코, 아일라 피셔, 저스티스 스미스, 도..."
2,극장판 체인소 맨: 레제편,애니메이션,100분,2025.09.24.,9.27,None
3,나혼자 프린스,코미디,116분,2025.11.19.,7.53,"이광수, 황하, 듀이 칸, 꾸 티 짜, 음문석, 배제기"
4,국보,드라마,175분,2025.11.19.,9.11,"요시자와 료, 요코하마 류세이, 타카하타 미츠키, 테라지마 시노부, 모리 나나, 쿠..."
5,프레데터: 죽음의 땅,SF,107분,2025.11.05.,8.62,"엘르 패닝, 디미트리우스 슈스터-콜로아마탕기"
6,퍼스트 라이드,코미디,116분,2025.10.29.,7.57,"강하늘, 김영광, 차은우, 강영석, 한선화"
7,8번 출구,스릴러,95분,2025.10.22.,8.36,"니노미야 카즈나리, 코치 야마토, 고마츠 나나"


In [125]:
df.dtypes
df['평점'] = df['평점'].astype('float')
df['상영시간'] = df['상영시간'].replace('분', '').astype(int)
df['개봉일'] = pd.to_datetime(df['개봉일'].str.replace('.', '-'))
df.dtypes


제목              object
장르              object
상영시간             int64
개봉일     datetime64[ns]
평점             float64
출연              object
dtype: object

In [ ]:
df1 = df.groupby('장르')['평점'].mean().sort_values(ascending=False)
print("장르별 평균 평점:")
print(df1)
print("\n")

장르별 평균 평점:
장르
애니메이션    9.27
드라마      9.11
SF       8.62
스릴러      8.36
범죄       7.83
코미디      7.55
판타지      7.54
Name: 평점, dtype: float64




In [128]:
df2 = df.groupby('장르')['상영시간'].mean().sort_values(ascending=False)
print("장르별 평균 상영시간:")
print(df2)
print("\n")

장르별 평균 상영시간:
장르
드라마      175.0
판타지      137.0
코미디      116.0
범죄       112.0
SF       107.0
애니메이션    100.0
스릴러       95.0
Name: 상영시간, dtype: float64




In [130]:
df3 = df['상영시간'].corr(df['평점'])
print(f"상영시간과 평점 간 상관계수: {df3:.2f}")

상영시간과 평점 간 상관계수: 0.13
